In [1]:
# Gestion des données et calculs
import pandas as pd
import numpy as np

# Utilitaires de système et de temps
import os
import time
import json

# Géocodage (Conversion d'adresses/noms en coordonnées)
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Cartographie et visualisation
import folium
from folium import LinearColormap
from folium import features

In [2]:
# Chargement et Préparation
# Le fichier clean_data.csv est chargé
df_annonces = pd.read_csv("clean_data.csv", sep=',', low_memory=False)

# Conversion de 'prix_m2' en numérique
df_annonces['prix_m2'] = pd.to_numeric(df_annonces['prix_m2'], errors='coerce')

# Nettoyage
# On conserve uniquement les lignes où 'prix_m2' et 'departement' sont valides
# On garde les lignes avec Address vide pour les départements
df_annonces.dropna(subset=['prix_m2', 'departement'], inplace=True)

In [3]:
# Géocodage des Départements Uniques

# Récupérer les départements uniques
departements_uniques = df_annonces['departement'].unique()
departements_coords = {}

# Configurer le géocodeur Nominatim avec timeout
geolocator = Nominatim(
    user_agent="votre_application_immobilier_script_final",
    timeout=5
)

# RateLimiter pour éviter les blocages par l'API
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.1)

print("Géocodage des départements :")

for departement in departements_uniques:
    # Ajouter "France" pour éviter les confusions internationales
    search_query = f"{departement}, France"
    try:
        # Limiter la recherche à la France
        location = geocode(
            search_query,
            country_codes='fr',
            exactly_one=True,
            addressdetails=True
        )

        if location and 'address' in location.raw:
            # Vérification que le lieu correspond à un département
            if 'state' in location.raw['address']:
                departements_coords[departement] = (
                    location.latitude,
                    location.longitude
                )
                print(
                    f"✅ Géocodé: {departement} -> "
                    f"({location.latitude}, {location.longitude})"
                )
            else:
                print(f"❌ Résultat non département: {departement}")
        else:
            print(f"❌ Non trouvé: {departement}")

    except Exception as e:
        # Gérer les erreurs de connexion / timeout
        print(
            f"⚠️ Erreur (Timeout/API) pour {departement}: {e}. "
            "Poursuite..."
        )
        time.sleep(2)

print("Fin du géocodage.")

Géocodage des départements :
✅ Géocodé: Ain -> (46.0652385, 5.2847717)
✅ Géocodé: Aisne -> (49.4532855, 3.606899)
✅ Géocodé: Allier -> (46.3674641, 3.1638828)
✅ Géocodé: Alpes-de-Haute-Provence -> (44.1640832, 6.1878515)
✅ Géocodé: Hautes-Alpes -> (44.6564666, 6.3520246)
✅ Géocodé: Alpes-Maritimes -> (43.9210587, 7.1790785)
✅ Géocodé: Ardèche -> (44.815194, 4.3986525)
✅ Géocodé: Ardennes -> (49.6980118, 4.6716005)
✅ Géocodé: Ariège -> (42.9455368, 1.4065544)
✅ Géocodé: Aube -> (48.3201921, 4.1905397)
✅ Géocodé: Aude -> (43.0542733, 2.5124715)
✅ Géocodé: Aveyron -> (44.3158575, 2.5065697)
✅ Géocodé: Bouches-du-Rhône -> (43.5424182, 5.0343236)
✅ Géocodé: Calvados -> (49.0907886, -0.241238)
✅ Géocodé: Cantal -> (45.0497701, 2.6997176)
✅ Géocodé: Charente -> (45.6667902, 0.097305)
✅ Géocodé: Charente-Maritime -> (45.7302268, -0.7212876)
✅ Géocodé: Cher -> (47.0248824, 2.5753334)
✅ Géocodé: Corrèze -> (45.3429047, 1.8176424)
✅ Géocodé: Côte-d'Or -> (47.4655034, 4.7481223)
✅ Géocodé: Côtes-d

In [4]:
# Géocodage des adresses précises pour Paris

# Filtrer par 'departement' == 'Paris'
# On suppose que cette colonne est une chaîne de caractères propre
df_paris_cp = df_annonces[
    (df_annonces['departement'] == 'Paris') & 
    (df_annonces['code_postal'].notna())
].copy()

# S'assurer que le code postal est une chaîne de caractères pour le regroupement
df_paris_cp['code_postal'] = df_paris_cp['code_postal'].astype(str)

# Ajou du mapping 75108 -> 75018
cp_mapping = {
    '75108': '75018',
}

df_paris_cp['code_postal'] = df_paris_cp['code_postal'].apply(
    lambda x: cp_mapping.get(x, x)
)

# Regrouper par code postal (arrondissement)
df_unique_cp_paris = df_paris_cp.groupby('code_postal').agg(
    prix_moyen_m2=('prix_m2', 'mean'),
    nombre_annonces=('code_postal', 'count')
).reset_index()

# Renommage pour clarté
df_unique_cp_paris.rename(columns={'code_postal': 'Code_Postal'}, inplace=True)

print(f"Nombre total d'annonces à Paris (filtrées) : {len(df_paris_cp)}")
print(f"Nombre de Codes Postaux UNIQUES (Arrondissements) à géocoder : {len(df_unique_cp_paris)}")

# Préparation pour stocker les résultats
coordonnees_cp_paris = []
successful_geocodes_cp = 0

# Configuration du géocodeur
geolocator_cp = Nominatim(
    user_agent="votre_application_immobilier_paris_codepostal",
    timeout=5
)
geocode_cp = RateLimiter(geolocator_cp.geocode, min_delay_seconds=1.1)

# Boucle de géocodage par code postal
for index, row in df_unique_cp_paris.iterrows():
    cp_original = row['Code_Postal']
    prix_moyen_m2 = row['prix_moyen_m2']
    nb_annonces = row['nombre_annonces']
    location = None

    # Requête : recherche uniquement sur le Code Postal
    search_query = f"{cp_original}, Paris, France"

    try:
        location = geocode_cp(search_query, country_codes='fr', exactly_one=True)
        
        if location:
            successful_geocodes_cp += 1
            print(f"✅ Réussi: CP {cp_original} -> ({location.latitude}, {location.longitude})")
        else:
            print(f"❌ Échec pour le CP: {cp_original}")

    except Exception as e:
        print(f"⚠️ Erreur (Timeout/API) pour CP {cp_original}: {e}. Poursuite...")
        time.sleep(2)

    # Enregistrer les coordonnées si trouvées
    if location:
        coordonnees_cp_paris.append({
            "latitude": location.latitude,
            "longitude": location.longitude,
            "prix_m2": prix_moyen_m2, 
            "nb_annonces": nb_annonces,
            "code_postal": cp_original
        })

# Conversion des résultats en DataFrame global pour les cartes
df_coordonnees_paris_arrondissements = pd.DataFrame(coordonnees_cp_paris)

print(f"\n✅ {successful_geocodes_cp} Codes Postaux (Arrondissements) géocodés pour Paris.")

Nombre total d'annonces à Paris (filtrées) : 31167
Nombre de Codes Postaux UNIQUES (Arrondissements) à géocoder : 23
✅ Réussi: CP 75000 -> (48.8522751, 2.3967596)
✅ Réussi: CP 75001 -> (48.8618779, 2.3374139)
✅ Réussi: CP 75002 -> (48.8676828, 2.3431278)
✅ Réussi: CP 75003 -> (48.8626858, 2.3586866)
✅ Réussi: CP 75004 -> (48.8541478, 2.3568079)
✅ Réussi: CP 75005 -> (48.8454189, 2.3525824)
✅ Réussi: CP 75006 -> (48.8493765, 2.3322544)
✅ Réussi: CP 75007 -> (48.854909, 2.3128667)
✅ Réussi: CP 75008 -> (48.8733792, 2.3111527)
✅ Réussi: CP 75009 -> (48.8770673, 2.3379172)
✅ Réussi: CP 75010 -> (48.8760081, 2.360445)
✅ Réussi: CP 75011 -> (48.8600808, 2.3781399)
✅ Réussi: CP 75012 -> (48.8352003, 2.4451359)
✅ Réussi: CP 75013 -> (48.8298445, 2.3630439)
✅ Réussi: CP 75014 -> (48.8295585, 2.323974)
✅ Réussi: CP 75015 -> (48.8414302, 2.2961649)
✅ Réussi: CP 75016 -> (48.854928, 2.2553299)
✅ Réussi: CP 75017 -> (48.8873906, 2.3064833)
✅ Réussi: CP 75018 -> (48.8916223, 2.3479768)
✅ Réussi: CP 

In [9]:
# Agrégation des données avec nombre d'annonces et préparation des GeoJSON

# Calculer le Prix Moyen et le Nombre d'annonces par département
df_departements = df_annonces.groupby('departement').agg(
    prix_moyen_m2=('prix_m2', 'mean'),
    nombre_annonces=('departement', 'count')
).reset_index()

# Préparation du GeoJSON
geojson_data = {
    "type": "FeatureCollection",
    "features": []
}

for index, row in df_departements.iterrows():
    departement = row['departement']
    prix_moyen = row['prix_moyen_m2']
    nb_annonces = row['nombre_annonces']

    if departement in departements_coords:
        lat, lon = departements_coords[departement]

        # Création de la Feature GeoJSON
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [lon, lat] 
            },
            "properties": {
                "departement": departement,
                "prix_moyen_m2": prix_moyen,
                "nombre_annonces": nb_annonces
            }
        }
        geojson_data['features'].append(feature)

# Sauvegarde du GeoJSON
geojson_file_path = 'departements_points.geojson'
with open(geojson_file_path, 'w', encoding='utf-8') as f:
    json.dump(geojson_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ GeoJSON de points de département créé: {geojson_file_path}")


# Création du GeoJSON des points d'annonces de Paris
geojson_paris_data = {
    "type": "FeatureCollection",
    "features": []
}

# Utilisation du nouveau DataFrame
for index, row in df_coordonnees_paris_arrondissements.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            # GeoJSON utilise [longitude, latitude]
            "coordinates": [row['longitude'], row['latitude']]
        },
        "properties": {
            # CHANGEMENT 2 : Remplacement de 'address' par 'code_postal'
            "code_postal": row['code_postal'],
            "prix_moyen_m2": row['prix_m2'],
            # CHANGEMENT 3 : Ajout de 'nombre_annonces'
            "nombre_annonces": row['nb_annonces']
        }
    }
    geojson_paris_data['features'].append(feature)

# Mise à jour du nom du fichier pour refléter l'agrégation
geojson_paris_file_path = 'paris_arrondissements_points.geojson'
with open(geojson_paris_file_path, 'w', encoding='utf-8') as f:
    # Assurez-vous d'avoir 'json' importé plus haut (import json)
    json.dump(geojson_paris_data, f, ensure_ascii=False, indent=2)

print(f"✅ GeoJSON des points d'arrondissements de Paris créé: {geojson_paris_file_path}")


✅ GeoJSON de points de département créé: departements_points.geojson
✅ GeoJSON des points d'arrondissements de Paris créé: paris_arrondissements_points.geojson


In [6]:
# Visualisation par Département avec Folium (2 cartes)

if df_departements.empty:
    print("ATTENTION : Aucune donnée pour générer les cartes.")
else:
    # Déterminer min et max pour la Colormap (Prix)
    prix_min = df_departements['prix_moyen_m2'].min()
    prix_max = df_departements['prix_moyen_m2'].max()
    
    # Déterminer min et max pour la taille du rayon (Nombre d'annonces)
    annonces_min = np.nanmin(df_departements['nombre_annonces'])
    annonces_max = np.nanmax(df_departements['nombre_annonces'])
    
    # Définition de la plage de rayon souhaitée pour la carte dynamique
    MIN_RADIUS = 5  
    MAX_RADIUS = 25 

    # Fonction de mise à l'échelle du rayon
    def scale_radius(value, min_val, max_val, min_r, max_r):
        """Mise à l'échelle linéaire de la valeur brute vers une plage de rayons (pixels)."""
        if max_val == min_val or pd.isna(value):
            return (min_r + max_r) / 2
        return min_r + (max_r - min_r) * (value - min_val) / (max_val - min_val)


    # Colormap discrète 5 niveaux
    from branca.colormap import LinearColormap
    colormap = LinearColormap(
        colors=['green', 'yellow', 'blue', 'purple', 'red'],
        vmin=prix_min,
        vmax=prix_max
    ).to_step(5)

# -----------------------------------------------------------
# CARTE 1 : Taille Dynamique (Radius = f(Nombre d'annonces))
# -----------------------------------------------------------

m_dynamique = folium.Map(location=[46.603354, 1.888334], zoom_start=6)
for feature in geojson_data['features']:
    lon, lat = feature['geometry']['coordinates']
    props = feature['properties']
    departement = props['departement']
    prix = props['prix_moyen_m2']
    nb_annonces = props['nombre_annonces']
    
    color = colormap(prix)
    
    # Calcul du rayon dynamique (Nombre d'annonces)
    radius = scale_radius(nb_annonces, annonces_min, annonces_max, MIN_RADIUS, MAX_RADIUS)
    popup_html = (
        f"<b style='font-size:16px'>Département : {departement}</b><br>"
        f"Prix Moyen/m² : {prix:,.0f} €<br>"
        f"Nombre d'annonces : {nb_annonces:,}"
    )
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=radius, # Taille dynamique
        color=color, 
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m_dynamique)
m_dynamique.add_child(colormap)
map_file_path_dynamique = 'carte_prix_moyen_dynamique.html'
m_dynamique.save(map_file_path_dynamique)
print(f"✅ Carte 1 générée et sauvegardée : {map_file_path_dynamique}")

# -----------------------------------------------------------
# Préparation à la carte 2 (PARIS)
# -----------------------------------------------------------

# Ajout des bornes d'annonces Paris
if 'df_coordonnees_paris_arrondissements' in locals() and not df_coordonnees_paris_arrondissements.empty:
    
    # Calculer les bornes min et max des prix/m² uniquement pour Paris (Arrondissements)
    prix_min_paris = df_coordonnees_paris_arrondissements['prix_m2'].min()
    prix_max_paris = df_coordonnees_paris_arrondissements['prix_m2'].max()
    
    # Utilisation de 'nb_annonces'
    annonces_min_paris = df_coordonnees_paris_arrondissements['nb_annonces'].min()
    annonces_max_paris = df_coordonnees_paris_arrondissements['nb_annonces'].max()

    
    # Redéfinition de la Colormap (Légende) pour la plage de prix de Paris
    colormap_paris = LinearColormap(
        colors=['green', 'yellow', 'blue', 'purple', 'red'], 
        vmin=prix_min_paris,
        vmax=prix_max_paris
    ).to_step(5)
    
    print(f"La Colormap de Paris est maintenant calibrée entre {prix_min_paris:,.0f}€ (vert) et {prix_max_paris:,.0f}€ (rouge).")
    
    # ----------------------------------------------
    # CARTE 2 : Détaillée des points d'annonces de Paris par arrondissement)
    # ----------------------------------------------
    
    df_paris_viz = df_coordonnees_paris_arrondissements.copy()
    
    m_paris = folium.Map(location=[48.8566, 2.3522], zoom_start=12)
    
    # Ajout des points par Code Postal (Arrondissement)
    for index, row in df_paris_viz.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        
        cp = row['code_postal']
        prix = row['prix_m2']
        # Utilisation de 'nb_annonces' dans la boucle
        nb_annonces = row['nb_annonces'] 
        
        # Utilisation de colormap_paris
        color = colormap_paris(prix)
        
        # Utilisation des bornes d'annonces de Paris (annonces_min_paris, annonces_max_paris)
        radius = scale_radius(nb_annonces, annonces_min_paris, annonces_max_paris, MIN_RADIUS, MAX_RADIUS)
        
        popup_html = (
            f"<b style='font-size:14px'>Arrondissement : {cp}</b><br>"
            f"Prix Moyen/m² : {prix:,.0f} €<br>"
            f"Nombre d'annonces: {nb_annonces:,}"
        )
        
        folium.CircleMarker(
            location=[lat, lon],
            radius=radius,
            color=color,
            weight=1,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(m_paris)
        
    # Ajout de la nouvelle légende colormap_paris à la carte
    m_paris.add_child(colormap_paris) 


    map_file_path_paris = 'carte_prix_paris_par_arrondissement.html'
    m_paris.save(map_file_path_paris)
    print(f"✅ Carte 2 générée et sauvegardée : {map_file_path_paris}")

else:
    print("ATTENTION : Le DataFrame df_coordonnees_paris_arrondissements est vide ou non défini. Carte de Paris non générée.")

✅ Carte 1 générée et sauvegardée : carte_prix_moyen_dynamique.html
La Colormap de Paris est maintenant calibrée entre 5,964€ (vert) et 17,806€ (rouge).
✅ Carte 2 générée et sauvegardée : carte_prix_paris_par_arrondissement.html


In [7]:
# Affichage de la Carte 1 France: Couleur = Prix/m², Taille = Nombre d'annonces
m_dynamique

In [8]:
# Affichage de la Carte 2 Paris (par arrondissement): Couleur = Prix/m², Taille = Nombre d'annonces
m_paris